In [455]:
import numpy as np
import pandas as pd

In [456]:
from sklearn.preprocessing import Imputer

In [457]:
train = pd.read_csv("/home/bogan/Загрузки/train.csv")
test = pd.read_csv("/home/bogan/Загрузки/val.csv")
unlabeled = pd.read_csv("/home/bogan/unlabeled.csv")

,objid,ra,dec,u_0,g_0,r_0,i_0,z_0,clean,rowc,...,g_5,r_5,i_5,z_5,u_6,g_6,r_6,i_6,z_6,class
0,78023,121.079395,-9.343143,25.46583,22.53827,21.52258,na,na,1,107.03430,...,-0.1840898,-0.05463289,na,na,6,5,6,5,2,0
1,67239,121.080753,-9.339397,23.64559,24.57786,21.76326,20.94348,21.52855,1,134.68570,...,0.3717134,-0.1285481,-0.1223638,na,6,8,1,1,0,0
2,48461,121.070386,-9.332728,24.88126,23.49473,22.23877,21.38574,21.51635,1,84.76380,...,0.004124804,0.01414719,-0.05156832,na,6,4,6,4,5,0
3,55564,121.083052,-9.334026,23.57384,21.42411,20.43948,19.99905,19.82959,1,177.05670,...,-0.08095889,na,-0.08930081,na,3,4,5,2,4,0
4,96180,121.063351,-9.325032,23.85167,20.25245,19.6067,19.18877,19.08231,1,65.37769,...,-0.03782843,-0.09712007,na,0.1064725,3,4,7,6,3,0


In [458]:
imp = Imputer()

/usr/local/lib/python3.5/dist-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [459]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [460]:
train = train.sample(frac=1)

In [461]:
test = test.sample(frac=1)

In [462]:
train[train == "na"] = np.nan

In [463]:
test[test == "na"] = np.nan

In [464]:
unlabeled[unlabeled == "na"] = np.nan

In [465]:
train.head()

,objid,ra,dec,u_0,g_0,r_0,i_0,z_0,clean,rowc,...,g_5,r_5,i_5,z_5,u_6,g_6,r_6,i_6,z_6,class
8126,39239,159.203573,8.030109,23.77129,24.10118,22.67376,21.68555,21.18952,0,738.8234,...,-0.8820492,-0.08590110000000001,-0.2764333,-0.1201795,3,3,5,2,4,0
23811,48984,140.472321,0.378885,18.99814,18.74376,18.81707,NaN,18.52016,1,134.4225,...,0.04555603,-0.001647749,0.02041021,-0.02505142,6,8,7,5,3,2
19455,42832,173.729416,-0.608692,23.22789,22.53772,21.07805,19.92483,19.34288,1,386.4522,...,-0.08746122,-0.1689961,0.006052183,-0.04830702,3,4,5,2,3,1
21178,99590,44.999621,-8.583322,19.18563,18.8202,18.64659,18.41356,18.41718,1,930.9224,...,0.06020081,-0.0100101,0.016995,NaN,6,6,2,7,0,2
15314,92529,159.667686,0.380990,19.64928,17.703039999999994,16.704710000000002,16.24454,15.87251,1,461.6502,...,0.008520246,-0.002563726,0.01017724,NaN,3,5,6,6,4,1


In [466]:
train = train.drop(['rowc', 'colc', 'ra', 'dec'], axis=1)

In [467]:
test = test.drop(['rowc', 'colc', 'ra', 'dec'], axis=1)

In [468]:
unlabeled = unlabeled.drop(['rowc', 'colc', 'ra', 'dec'], axis=1)

In [469]:
data = train.iloc[:, 1:39]

In [470]:
data.head()

,u_0,g_0,r_0,i_0,z_0,clean,rowv,colv,u_1,g_1,...,u_5,g_5,r_5,i_5,z_5,u_6,g_6,r_6,i_6,z_6
8126,23.77129,24.10118,22.67376,21.68555,21.18952,0,0.0,0.0,746.3627,750.3187,...,NaN,-0.8820492,-0.08590110000000001,-0.2764333,-0.1201795,3,3,5,2,4
23811,18.99814,18.74376,18.81707,NaN,18.52016,1,0.002537426,-6.033498e-05,140.5313,145.8292,...,-0.01845432,0.04555603,-0.001647749,0.02041021,-0.02505142,6,8,7,5,3
19455,23.22789,22.53772,21.07805,19.92483,19.34288,1,-0.01867258,-0.006765311,391.5220,396.4805,...,-0.3861437,-0.08746122,-0.1689961,0.006052183,-0.04830702,3,4,5,2,3
21178,19.18563,18.8202,18.64659,18.41356,18.41718,1,0.0009875959,0.0007527082000000001,938.8677,943.0204,...,-0.008006225,0.06020081,-0.0100101,0.016995,NaN,6,6,2,7,0
15314,19.64928,17.703039999999994,16.704710000000002,16.24454,15.87251,1,0.0017282139999999998,0.001396771,467.4111,473.0844,...,NaN,0.008520246,-0.002563726,0.01017724,NaN,3,5,6,6,4


In [471]:
unlabeled = unlabeled.drop('objid', axis=1)

In [472]:
y = train.iloc[:, 39]

In [473]:
def norm_to_gauss(df):
    return (df - df.mean()) * 1.0 / (df.var())

In [474]:
y

8126     0
23811    2
19455    1
21178    2
15314    1
16442    1
10751    1
17891    1
22477    2
1563     0
24204    2
8485     0
13293    1
6303     0
7426     0
21847    2
17233    1
19798    1
8391     0
28076    2
4689     0
22728    2
19937    1
8053     0
11555    1
14580    1
22723    2
17662    1
6837     0
10872    1
        ..
21751    2
9902     0
20471    2
25472    2
29566    2
28651    2
21682    2
21197    2
10866    1
28697    2
4529     0
14292    1
4615     0
6051     0
21326    2
2971     0
12205    1
29056    2
1787     0
5428     0
24108    2
29935    2
4144     0
18638    1
14993    1
14434    1
26145    2
23600    2
25256    2
17738    1
Name: class, Length: 30000, dtype: int64

In [475]:
data = data.apply(np.float64)

In [440]:
data

,u_0,g_0,r_0,i_0,z_0,clean,rowv,colv,u_1,g_1,...,u_5,g_5,r_5,i_5,z_5,u_6,g_6,r_6,i_6,z_6
19656,NaN,-0.021346,0.107728,0.229906,0.213979,1.213306,-0.714438,0.611367,0.000077,0.000078,...,-0.000089,NaN,-0.002132,0.006088,1.900974e-04,0.717256,0.863337,0.129878,0.429368,-0.660803
18109,NaN,NaN,0.147031,NaN,NaN,1.213306,2.966170,-3.271866,0.004199,0.004189,...,-0.000089,0.000182,-0.001088,-0.017080,-4.299738e-05,0.717256,0.521769,-0.574788,0.942731,-0.660803
15078,-0.826389,-0.903239,-1.041710,-1.130815,-1.157320,1.213306,-0.005908,-0.344023,-0.002877,-0.002869,...,-0.000124,0.000066,-0.003214,-0.025275,-2.230669e-04,-0.619910,-0.502937,0.129878,0.172687,0.765929
20683,0.030377,0.005205,0.203824,0.319150,0.485641,1.213306,-3.352012,-1.604023,-0.001142,-0.001142,...,-0.000114,0.000358,-0.001059,NaN,-7.332535e-04,-0.619910,0.863337,0.834543,0.172687,0.765929
20637,-0.158912,NaN,-0.229158,-0.107231,-0.068958,1.213306,-2.650403,-3.168992,0.002532,0.002533,...,-0.000153,0.000189,NaN,0.004772,2.926214e-04,0.717256,0.521769,0.364766,0.172687,-1.017486
26957,NaN,-0.106571,0.032670,0.088088,0.053700,1.213306,0.117262,-0.973525,-0.000773,-0.000781,...,-0.000106,0.000234,-0.001426,NaN,5.201062e-04,-0.619910,0.180200,0.599655,0.429368,-1.017486
23967,-0.602390,-0.418322,-0.309843,-0.227807,-0.305236,1.213306,-0.401940,-0.366356,0.002269,0.002274,...,-0.000103,0.000213,-0.001184,-0.000873,-1.301170e-04,0.717256,0.863337,-0.105011,0.686049,-0.660803
19775,0.735227,-0.017105,-0.269293,-0.350864,-0.404646,1.213306,0.267290,-2.401034,-0.000262,-0.000249,...,0.000056,0.000192,NaN,0.002161,7.757555e-04,0.717256,-0.502937,0.129878,0.172687,0.765929
19520,-0.640930,NaN,-0.668585,NaN,NaN,1.213306,1.942080,-4.748111,0.002439,0.002438,...,NaN,0.000636,0.003955,0.072913,1.323539e-03,-0.619910,-0.502937,-0.105011,0.429368,0.765929
23831,-0.848484,NaN,-0.630718,-0.583698,-0.517365,1.213306,0.593146,-0.901861,0.003153,0.003141,...,-0.000079,0.000278,NaN,-0.009663,3.605598e-04,0.717256,0.180200,0.364766,-0.340676,-0.660803


In [476]:
data_test = test.iloc[:, 1:39]

In [477]:
data_test = data_test.apply(np.float64)

In [478]:
unlabeled = unlabeled.apply(np.float64)

In [479]:
result =  pd.concat([data, unlabeled, data_test])

In [480]:
result = result.apply(norm_to_gauss)

In [481]:
result = imp.fit_transform(result)

In [482]:
result.shape

(76666, 38)

In [581]:
type(result)

numpy.ndarray

In [483]:
data = result[:30000, :]

In [484]:
data.shape

(30000, 38)

In [485]:
unlabeled = result[30000:53333, :]

In [486]:
unlabeled.shape

(23333, 38)

In [487]:
data_test = result[53333:76666,: ]

In [488]:
data_test.shape

(23333, 38)

In [489]:
true = test['class']

In [191]:
true

21716    2
10562    0
10804    0
16287    1
19027    2
3793     0
15893    1
14527    1
3314     0
9710     0
22069    2
6934     0
12633    0
16515    1
20059    2
9075     0
1099     0
20358    2
16673    1
47       0
22676    2
7437     0
20855    2
8699     0
12100    0
16849    1
12036    0
5834     0
18557    2
7242     0
        ..
1851     0
19059    2
13163    0
8997     0
7483     0
19548    2
3021     0
7707     0
6262     0
21379    2
22492    2
5830     0
5436     0
10681    0
16863    1
12472    0
12712    0
6605     0
4839     0
6860     0
12196    0
1140     0
14817    1
738      0
14537    1
2559     0
8374     0
18053    1
20883    2
13138    0
Name: class, Length: 23333, dtype: int64

,u_0,g_0,r_0,i_0,z_0,clean,rowv,colv,u_1,g_1,...,u_5,g_5,r_5,i_5,z_5,u_6,g_6,r_6,i_6,z_6
17520,0.029432,-0.225889,-0.465556,-0.549366,-0.601093,1.213306,1.032454,3.096279,-0.003456,-0.003455,...,-0.000103,0.000159,-0.002153,-0.014743,0.000000,-0.619910,-0.502937,-0.574788,-0.083995,0.409246
26486,-0.305228,0.000000,-0.309046,0.000000,-0.192570,1.213306,-1.247701,-1.434410,0.001451,0.001447,...,-0.000093,0.000212,-0.000987,0.001830,0.000027,0.717256,0.863337,-0.339899,-0.083995,-1.017486
29227,-0.079392,-0.003655,0.000000,0.343537,0.377345,1.213306,2.421566,-2.285320,-0.000320,-0.000305,...,-0.000030,0.000259,-0.001523,0.006474,0.000484,0.717256,0.180200,-0.339899,0.429368,0.765929
13914,0.245642,0.018272,-0.146441,-0.241091,-0.253020,1.213306,6.048057,-9.790169,0.002314,0.002311,...,0.001177,0.000357,-0.000491,0.018959,0.000000,-0.619910,-0.502937,0.364766,-0.854039,0.052563
18121,0.000000,-0.529405,-0.521476,-0.489985,-0.437485,1.213306,-0.498177,-1.214749,-0.000342,-0.000333,...,-0.000091,0.000232,-0.001390,0.001672,0.000300,0.717256,0.863337,-0.105011,0.942731,-0.660803


In [490]:
def argmax(preds):
    return np.asarray([np.argmax(line) for line in preds])

In [491]:
from keras.utils.np_utils import to_categorical   

cl = to_categorical(y, num_classes=3)

In [492]:
from keras.layers.advanced_activations import *

In [493]:
from keras import regularizers

In [494]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
model = Sequential()

model.add(Dense(128,  input_dim=38, activation=PReLU()))
model.add(Dropout(0.4))
model.add(Dense(128, activation=PReLU()))
model.add(Dropout(0.3))
model.add(Dense(256, activation=PReLU()))
model.add(Dropout(0.3))
model.add(Dense(1024,activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(256,activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(128,activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(64,activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(9,  activation=PReLU()))
model.add(Dense(3,  activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=[f1])
hist = model.fit(data, cl, epochs=50, batch_size=1000)

predictions = argmax(model.predict(unlabeled))



/usr/local/lib/python3.5/dist-packages/keras/activations.py:115: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/50
30000/30000 [==============================] - 5s 160us/step - loss: 0.6303 - f1: 0.2573
Epoch 2/50
30000/30000 [==============================] - 2s 69us/step - loss: 0.4946 - f1: 0.5636
Epoch 3/50
30000/30000 [==============================] - 2s 69us/step - loss: 0.4139 - f1: 0.7316
Epoch 4/50
30000/30000 [==============================] - 2s 69us/step - loss: 0.3930 - f1: 0.7502
Epoch 5/50
30000/30000 [==============================] - 2s 69us/step - loss: 0.3818 - f1: 0.7618
Epoch 6/50
30000/30000 [==============================] - 2s 69us/step - loss: 0.3707 - f1: 0.7699
Epoch 7/50
30000/30000 [==============================] - 2s 71us/step - loss: 0.3612 - f1: 0.7772
Epoch 8/50
30000/30000 [==============================] - 2s 70us/step - loss: 0.3569 - f1: 0.7833
Epoch 9/50
30000/30000 [==============================] - 2s 70us/step - loss: 0.3529 - f1: 0.7855
Epoch 10/50
30000/30000 [==============================] - 2s 70us/step - loss: 0.3478 - f1: 0.7898
Epoch 11

In [495]:
from sklearn.metrics import f1_score

In [496]:
xg = [*y, *predictions]

In [497]:
result = np.concatenate((data, unlabeled), axis=0)

In [498]:
import lightgbm as lgb

dtrain = lgb.Dataset(result, label=xg)



In [499]:
parameters = { 
    'boosting': 'dart',
    'objective': 'softmax',
    'num_class': 3, 
    'max_depth': 50,
    'num_leaves': 70,
    'bagging_freq': 1000,
    'bagging_fraction': 0.8,
    'bagging_seed': 42,
    'feature_fraction': 0.8,
    'drop_rate': 0,
    'max_drop': 10,
    'uniform_drop': True,
    'xgboost_dart_mode': True
    }  
num_round = 300

In [500]:
bst = lgb.train(parameters, dtrain, num_round)

In [501]:
preds = bst.predict(data_test)

In [502]:
yhat = argmax(preds)

In [503]:
f1_score(true, yhat, average="macro")

0.859047393850867

In [44]:
from sklearn.preprocessing import PolynomialFeatures as pf

In [223]:
poly = pf(interaction_only=True)
r = poly.fit_transform(data)
u = poly.fit_transform(unlabeled)


In [187]:
from sklearn.ensemble import RandomForestClassifier

In [360]:
clf = RandomForestClassifier(max_depth=500, random_state=0, criterion = "entropy")
clf.fit(result, xg)

h =clf.predict(data_test)


In [361]:
f1_score(true, h, average="macro")

0.8385039692524598

In [47]:
predictions

array([0, 0, 0, ..., 0, 0, 1])

In [48]:
true = test['class']

In [49]:
np.unique(predictions)

array([0, 1, 2])

In [316]:
xg = [*y, *predictions]

In [27]:

df = pd.DataFrame(data={'objid': test.iloc[:, 0], 'prediction': predictions})

In [29]:
df.to_csv('predictions.csv', encoding='utf-8', index=False)

In [362]:
cl1 = to_categorical(predictions, num_classes=3)

In [363]:
res1 = np.concatenate((cl, cl1), axis=0)

In [56]:
res1.shape

(53333, 3)

In [317]:
result = np.concatenate((data, unlabeled), axis=0)

In [52]:
result.shape

(53333, 38)

In [411]:
result.shape

(53333, 38)

In [415]:
res.shape

(53333, 3)

In [241]:
unlabeled.shape

(23333, 38)

In [242]:
data.shape

(30000, 38)

In [243]:
y.shape

(30000,)

In [244]:
predictions.shape

(23333,)

In [245]:
res.shape

(60000, 3)

In [246]:
result.shape

(53333, 38)

In [344]:
y = [*y, *predictions]

In [347]:
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)


In [349]:
forest.fit(result_set, y)
importances = forest.feature_importances_

In [351]:
len(importances)

38

In [359]:
np.sort(forest.feature_importances_)

array([0.01764135, 0.01765006, 0.01768147, 0.0177305 , 0.0177377 ,
       0.01791236, 0.0179544 , 0.01796203, 0.01796489, 0.01800402,
       0.01898066, 0.01921714, 0.01939224, 0.0198768 , 0.02023242,
       0.02059063, 0.02084249, 0.02097165, 0.02099698, 0.02107279,
       0.02120809, 0.02147644, 0.02190807, 0.02272362, 0.02378905,
       0.024819  , 0.02673036, 0.027428  , 0.02827767, 0.03016341,
       0.03536408, 0.04056061, 0.04372862, 0.04583848, 0.04639223,
       0.04731027, 0.05528205, 0.05658739])

In [229]:
from sklearn.ensemble import RandomForestClassifier

In [285]:
clf = RandomForestClassifier(max_depth=20, random_state=0, criterion="entropy")
clf.fit(result, y)




RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [286]:
importance = dict(zip([i for i in range(1, 743)], [*clf.feature_importances_]))

In [287]:
importance = sorted(importance.items(), key=lambda x: x[1])

In [288]:
importance

[(8, 0.0),
 (1, 0.00772060926465354),
 (9, 0.008596884477528727),
 (20, 0.008676288498310081),
 (14, 0.009522736327961434),
 (6, 0.0103362089835594),
 (16, 0.01295744705622042),
 (21, 0.01303609120897765),
 (17, 0.013730273655273536),
 (19, 0.013819187844340585),
 (24, 0.014197624730623016),
 (22, 0.014665192661355719),
 (18, 0.014706852755501598),
 (27, 0.015598171422107352),
 (28, 0.016310825316650474),
 (29, 0.01797653138890159),
 (32, 0.01969502283789113),
 (31, 0.0208898596147152),
 (26, 0.022809709840223014),
 (33, 0.022834982773918346),
 (34, 0.025534162221620915),
 (30, 0.026045039913697908),
 (25, 0.028039138551970454),
 (23, 0.030714415666479734),
 (35, 0.033234486794944104),
 (36, 0.03677046484078829),
 (10, 0.039825489509067245),
 (5, 0.04136528752329199),
 (12, 0.04350532458027504),
 (4, 0.04458178612614732),
 (11, 0.047948429426544106),
 (13, 0.04919966362883012),
 (3, 0.05644092621310952),
 (2, 0.06252080223432002),
 (7, 0.07637665138168918),
 (15, 0.07981743072851122)]

In [297]:
h = clf.predict(c)

In [293]:
lr=data_test[:, [35, 4, 2, 20, 3, 5, 1]]

In [296]:
c= np.concatenate([lr, poly.fit_transform(lr)], axis=1)

In [271]:
data1 = data[:, [35, 4, 2, 20, 3, 5, 1]]

In [272]:
data2 = poly.fit_transform(data1)

In [274]:
result = np.concatenate([data1, data2], axis=1)


In [298]:
f1_score(true, h, average="macro")

0.791666449715811

In [523]:
train

,objid,u_0,g_0,r_0,i_0,z_0,clean,rowv,colv,u_1,...,g_5,r_5,i_5,z_5,u_6,g_6,r_6,i_6,z_6,class
12629,65963,24.70814,23.61093,21.26833,19.91146,19.1904,1,-0.01123665,0.006698497,789.30260,...,0.09144247,0.03335264,0.07488435,NaN,3,6,1,1,4,1
9913,53873,24.158620000000006,23.6499,22.39743,NaN,21.39384,1,0.02013935,0.1494356,1292.79700,...,-0.4641099,-0.15072249999999998,0.1080337,-0.1461001,3,3,7,2,3,0
27390,21420,19.80087,19.5241,19.15802,18.93851,18.68839,1,-0.00013271030000000002,-0.0008919533999999998,458.36190,...,0.02293431,NaN,0.006275014000000001,0.07282342,6,8,8,5,0,2
8414,59024,23.62051,22.60194,22.3207,21.47172,21.10028,0,0.0,0.0,1385.36800,...,-0.1928688,-0.01350927,0.0773791,-7.482324,3,4,6,1,3,0
29885,32768,21.66067,20.9472,21.02726,21.15996,20.38695,1,-0.001763075,-0.0004566405,948.35520,...,0.03914404,-0.04824586,-0.09059825,NaN,3,6,7,7,2,2
20383,81629,NaN,20.85205,20.278,20.26909,19.98174,1,0.0003610044,-0.002105455,414.27840,...,-0.009495582,-0.07069572,-0.1091296,0.04793119,3,7,8,7,2,2
2544,81420,NaN,22.1766,21.05435,20.86696,20.01108,1,0.01095406,0.001743299,1159.79900,...,-0.07740784,0.06296306,0.02360508,0.06813441,6,3,4,5,2,0
23229,89410,23.94407,21.91046,21.5843,21.49489,21.1067,1,-0.005772434,0.0007036537,1242.36800,...,0.08979928,NaN,0.35132129999999995,NaN,3,6,5,5,1,2
20892,83998,22.0571,20.89088,20.72654,20.76922,20.32292,1,0.0005441257,0.0035593490000000007,1072.51300,...,-0.03445388,-0.05221053,0.09388769,-0.05851971,6,8,4,8,4,2
21611,88017,20.18942,19.64664,NaN,18.46525,18.23145,1,0.002222208,0.002072365,801.22300,...,0.002161361,0.02259567,0.00995411,NaN,6,4,3,1,3,2


In [527]:
unlabeled = result_set.iloc[30000:, ]

In [528]:
data=result_set.iloc[:30000, ]

In [529]:
data.shape

(30000, 38)